# Method

## Preprocessing data
The raw data is supplied by Datavärde ([datavarde.se](https://datavarde.se/)) as folder structure with a series of zipped json files for each measured item (''telematikenhet''). The measured items are resampled with 1 second sampling period to enable time syncing between the items, so that all items can be represented in one time series table for each vessel.  

## Cleaning data
* Latitudes should be in intervall -90  to 90 degrees.
* Longitudes should be in the intervall 0 to 180 degrees.
* Valued smaller than 0.001 percentile for *sog* and *fuel rate* are removed
* Valued larger than 0.999 percentile for *sog* and *fuel rate* are removed


## Selection data
The following selections are defined:
* *all*: all data points
* *moving*: data points where the ships are moving (sog > 0) 
* *running*: data points where the ships are running (sog > 2 kts)

## Statistics

The average course over ground $\overline{cog}$ is calculated as: 

```{math}
:label: eq-cog

\overline{cog} = \arctan \left(\frac{\overline{\sin(cog)}}{\overline{\cos(cog)}} \right)
```
where line over means arithmetic mean value.

$$ \overline{cog} = \arctan \left(\frac{\overline{\sin(cog)}}{\overline{\cos(cog)}} \right)$$

The distance and consumption are calculated as the time integral:

```{math}
:label: eq-integral

x_{total} = \int_{n=0}^{n=N} x_n \,dt
```

$$ x_{total} = \int_{n=0}^{n=N} x_n \,dt $$

The integral is solved with simpson integration where $x$ is replaced by *sog* or *fuel rate/3600* to get distance or consumption. The sampling period is used for $dt$. The integration is split into many integrations if there are gaps in the time signal:

```{math}
:label: eq-integral-split

x_{total} = \int_{n=0}^{n=N} x_n \,dt + \int_{n=N+1}^{n=K} x_n \,dt + ...
```

$$ x_{total} = \int_{n=0}^{n=N} x_n \,dt + \int_{n=N+1}^{n=K} x_n \,dt + ... $$

## Fuel consumption model

A spline function is used as the model to describe the fuel consumption as function of sog. The number of knots and degree of the spline is selected with Repeated K-fold cross validation. 